In [ ]:
import torch
import torchaudio
import k2
import icefall
import matplotlib.pyplot as plt

In [40]:
# bundle = torchaudio.pipelines.WAV2VEC2_XLSR_300M
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
model = bundle.get_model()

print("Sample Rate:", bundle.sample_rate)
print("Labels:", bundle.get_labels())

Sample Rate: 16000
Labels: ('-', '|', 'E', 'T', 'A', 'O', 'N', 'I', 'H', 'S', 'R', 'D', 'L', 'U', 'M', 'W', 'C', 'F', 'G', 'Y', 'P', 'B', 'V', 'K', "'", 'X', 'J', 'Q', 'Z')


In [ ]:
wav_orig, orig_freq = torchaudio.load("/home/mjsimmons/projects/malachor5/test/data/albrrizo.wav")
new_freq = 16_000
resample_f = torchaudio.transforms.Resample(orig_freq=orig_freq, new_freq=new_freq)
wav = resample_f(wav_orig)

torch.Size([2, 17798])

In [36]:
with torch.inference_mode():
    features, _ = model.extract_features(wav)

In [ ]:
fig, ax = plt.subplots(len(features), 1, figsize=(16, 4.3 * len(features)))
for i, feats in enumerate(features):
    ax[i].imshow(feats[0].cpu(), interpolation="nearest")
    ax[i].set_title(f"Feature from transformer layer {i+1}")
    ax[i].set_xlabel("Feature dimension")
    ax[i].set_ylabel("Frame (time-axis)")
fig.tight_layout()


In [38]:
with torch.inference_mode():
    emission, _ = model(wav)

In [39]:
emission.shape

torch.Size([2, 55, 29])

Following along `test_ctc_topo.py` in [Icefall github repo](https://github.com/k2-fsa/icefall/blob/master/icefall/ctc/test_ctc_topo.py)

In [48]:
from icefall.ctc.topo import build_standard_ctc_topo
import kaldifst

H=build_standard_ctc_topo(max_token_id=len(bundle.get_labels()))

input_symbols = kaldifst.SymbolTable()
output_symbols = kaldifst.SymbolTable()

input_symbols.add_symbol(symbol="<blk>", key=0)
output_symbols.add_symbol(symbol="<eps>", key=0)

for i, label in enumerate(bundle.get_labels(), start=1):
    input_symbols.add_symbol(symbol=label, key=i)
    output_symbols.add_symbol(symbol=label, key=i)

H.input_symbols = input_symbols
H.output_symbols = output_symbols
